In [8]:
import os
import requests

def download_pixabay_images(query, num_images=10, output_dir="downloads"):
    """
    Download images from Pixabay based on a search query.

    Args:
        query (str): Search term, e.g., "puppies"
        num_images (int): Number of images to download
        output_dir (str): Directory to save downloaded images
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Pixabay API endpoint
    url = "https://pixabay.com/api/"

    # Parameters for the API request
    params = {
		"key": "YOUR_REAL_API_KEY",  # <-- replace this with your actual key
		"q": query,
		"image_type": "photo",
		"per_page": num_images
	}

    try:
        # Send GET request to Pixabay API
        response = requests.get(url, params=params)
        response.raise_for_status()  # Check if the request was successful

        # Parse the JSON response
        data = response.json()

        # Check if the response contains hits (images)
        if "hits" in data:
            for i, hit in enumerate(data["hits"]):
                # Get the image URL
                img_url = hit["largeImageURL"]

                # Send GET request to download the image
                img_response = requests.get(img_url)
                img_response.raise_for_status()  # Check if the request was successful

                # Define the image file path
                img_path = os.path.join(output_dir, f"{query}_{i+1}.jpg")

                # Save the image to the specified path
                with open(img_path, "wb") as img_file:
                    img_file.write(img_response.content)

                print(f"Downloaded: {img_path}")
        else:
            print("No images found for the given query.")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Example usage
download_pixabay_images("puppies", num_images=10, output_dir="puppies")


An error occurred: 400 Client Error: Bad Request for url: https://pixabay.com/api/?key=YOUR_REAL_API_KEY&q=puppies&image_type=photo&per_page=10


In [4]:
from ultralytics import YOLO
import cv2
import torch
from torchvision import models, transforms
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pretrained YOLO for people detection
detector = YOLO("yolov8n.pt")  # nano version, super fast

# Load pretrained CNN for feature extraction
feature_extractor = models.mobilenet_v3_small(pretrained=True).features.eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def get_features(img):
    with torch.no_grad():
        x = transform(img).unsqueeze(0)
        feats = feature_extractor(x).mean([2,3]).numpy()
    return feats

def analyze_image(image_path):
    img = cv2.imread(image_path)
    results = detector(img)
    boxes = results[0].boxes.xyxy.cpu().numpy()

    features = []
    for (x1, y1, x2, y2) in boxes:
        crop = img[int(y1):int(y2), int(x1):int(x2)]
        feats = get_features(transforms.ToPILImage()(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)))
        features.append(feats)
    
    print(f"Detected {len(boxes)} person(s)")
    return np.vstack(features)

# Example
feats = analyze_image("test.jpg")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/ythacker/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8n.pt... <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1018)>


######################################################################## 100.0%
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /Users/ythacker/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1018)>